## Libraries

pd - used to transform data from/to dataframes in order to preprocess them

In [1]:
import pandas as pd

## Helpers

We use configuration table used by SIM-TB to refer to integer options and values in the main tables

In [2]:
configurationDF = pd.read_csv('../data/configurations.csv')
def configureNominalValues(column):
    configuration = {}
    for index, row in configurationDF.iterrows():
        if row['Category'] == column:
            configuration[row['CategoryValue']] = row['CategoryOption']

    return configuration

## Patients

### Data Preparation:
1. Check for Null Values
2. Get Nominal Values from Configuration
3. Transform Registration Date from Datetime to Date 

In [3]:
patientsDF = pd.read_csv('../data/patients.csv')

In [4]:
patientsDF.isnull().sum()

ID                  0
CountyID            0
RegistrationDate    0
Age                 0
Gender              0
dtype: int64

In [5]:
configurationDF['Category'].replace(['KomunatLista'], 'CountyID', inplace=True)
for column in ['CountyID']:
    for key, value in configureNominalValues(column).items():
        patientsDF[column] = patientsDF[column].replace([key], value)
patientsDF

,ID,CountyID,RegistrationDate,Age,Gender
0,323,Ferizaj,2018-04-10 10:12:48,48,M
1,325,Ferizaj,2018-04-10 10:53:23,72,M
2,326,Prishtinë,2018-04-13 7:05:48,50,F
3,327,Prishtinë,2018-04-10 12:01:03,75,F
4,329,Prishtinë,2018-04-10 12:20:55,38,M
...,...,...,...,...,...
2711,4331,Gjilan,2022-11-30 9:00:04,78,F
2712,4332,Viti,2022-12-01 11:32:59,63,F
2713,4333,Skënderaj,2022-12-01 12:58:58,40,M
2714,4334,Gjilan,2022-12-02 8:17:14,0,F


In [6]:
patientsDF['RegistrationDate'] = pd.to_datetime(patientsDF['RegistrationDate']).dt.date

In [7]:
patientsDF

,ID,CountyID,RegistrationDate,Age,Gender
0,323,Ferizaj,2018-04-10,48,M
1,325,Ferizaj,2018-04-10,72,M
2,326,Prishtinë,2018-04-13,50,F
3,327,Prishtinë,2018-04-10,75,F
4,329,Prishtinë,2018-04-10,38,M
...,...,...,...,...,...
2711,4331,Gjilan,2022-11-30,78,F
2712,4332,Viti,2022-12-01,63,F
2713,4333,Skënderaj,2022-12-01,40,M
2714,4334,Gjilan,2022-12-02,0,F


In [8]:
patientsDF.to_csv('../data/patients_processed.csv', index=False)

## Cases

### Data Preperation
1. Check for Null Values
2. Transform RegistrationDate from datetime to date
2. One Hot Encode Risk Factors
3. Get Nominal Values from Configuration

In [26]:
casesDF = pd.read_csv('../data/cases.csv')

In [27]:
casesDF.isnull().sum()

ID                     0
PatientID              0
Class                  0
Placement              0
RiskFactors         2612
RegistrationDate       0
TreatementDate         0
dtype: int64

In [29]:
casesDF['RegistrationDate'] = pd.to_datetime(casesDF['RegistrationDate']).dt.date

In [30]:
casesDF['RiskFactors'].unique()

array(['Droga, Duhani', 'Diabeti', nan, 'Neoplasia', 'Duhani',
       'Steroidet dhe terapia imunosupresive', 'Alkooli, Duhani',
       'Diabeti,,', 'Duhani,', 'Diabeti, Ulcera e lukthit',
       'Ulcera e lukthit', 'Diabeti, Duhani', 'HIV', 'Alkooli'],
      dtype=object)

In [31]:
casesDF['RiskFactors'].fillna('None', inplace=True)

In [32]:
one_hot_encoding_columns = []
for risk in casesDF['RiskFactors'].unique():
    if risk != 'None':
      for risk_factor in risk.split(','):
        if risk_factor not in one_hot_encoding_columns and risk_factor != '':
          one_hot_encoding_columns.append(risk_factor.strip())
one_hot_encoding_columns

['Droga',
 'Duhani',
 'Diabeti',
 'Neoplasia',
 'Steroidet dhe terapia imunosupresive',
 'Alkooli',
 'Duhani',
 'Ulcera e lukthit',
 'Duhani',
 'HIV']

In [33]:
for column in one_hot_encoding_columns:
  for index, row in casesDF.iterrows():
    if row['RiskFactors'] == 'None':
      casesDF.loc[index, column] = 0
    elif column in row['RiskFactors']:
      casesDF.loc[index, column] = 1
    else:
      casesDF.loc[index, column] = 0

In [ ]:
casesDF

In [35]:
configurationDF['Category'].replace(['LokalizimiLista'], 'Class', inplace=True)
configurationDF['Category'].replace(['ExtraPulmunareSmundjaVendi'], 'Placement', inplace=True)
for column in ['Class', 'Placement']:
    for key, value in configureNominalValues(column).items():
        casesDF[column] = casesDF[column].replace([key], value)
casesDF

,ID,PatientID,Class,Placement,RiskFactors,RegistrationDate,TreatementDate,Droga,Duhani,Diabeti,Neoplasia,Steroidet dhe terapia imunosupresive,Alkooli,Ulcera e lukthit,HIV
0,320,323,Ex-Pulmonar,Eshtra,"Droga, Duhani",2018-04-10,2018-01-16 0:00:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,322,325,Pulmonar,0,Diabeti,2018-04-10,2018-01-11 0:00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,323,326,Pulmonar,0,None,2018-04-10,2017-12-13 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,324,327,Pulmonar,0,None,2018-04-10,2017-12-07 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,326,329,Pulmonar,0,None,2018-04-10,2017-12-07 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,4324,4331,Ex-Pulmonar,Gj. limfatike,None,2022-11-30,2022-11-29 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2713,4325,4332,Pulmonar,0,None,2022-12-01,2022-11-30 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2714,4326,4333,Pulmonar,0,None,2022-12-01,2022-09-30 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2715,4327,4334,Ex-Pulmonar,0,None,2022-12-02,2022-12-01 0:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
casesDF.to_csv('../data/cases_processed.csv', index=False)

## Examinations

### Data Preparation
1. Check Null Values
2. Transform RegistrationDate from datetime to date
2. Get Nominal Values
3. Get Result Values
4. Remove Noisy Result Values

In [38]:
examinationsDF = pd.read_csv('../data/examinations.csv')
examinationsDF

,ID,CaseID,RegistrationDate,ExaminationType,ExaminationResult
0,476,320,2018-12-12 0:22:07,7,3
1,481,322,2018-04-10 10:37:06,1,2
2,482,322,2018-04-10 10:37:38,4,17
3,484,322,2018-04-10 10:49:20,1,2
4,485,323,2018-04-10 11:41:56,1,2
...,...,...,...,...,...
22322,26512,4329,2022-12-02 12:35:46,6,2
22323,26513,4329,2022-12-02 12:36:04,2,8
22324,26514,4329,2022-12-02 12:36:49,7,3
22325,26515,4329,2022-12-02 12:37:09,9,15


In [39]:
examinationsDF.isnull().sum()

ID                   0
CaseID               0
RegistrationDate     0
ExaminationType      0
ExaminationResult    0
dtype: int64

In [40]:
examinationsDF['RegistrationDate'] = pd.to_datetime(examinationsDF['RegistrationDate']).dt.date

In [41]:
configurationDF['Category'].replace(['EkzaminimiLlojiTestit'], 'ExaminationType', inplace=True)
for column in ['ExaminationType']:
    for key, value in configureNominalValues(column).items():
        examinationsDF[column] = examinationsDF[column].replace([key], value)
examinationsDF

,ID,CaseID,RegistrationDate,ExaminationType,ExaminationResult
0,476,320,2018-12-12,Histologjia,3
1,481,322,2018-04-10,Mikroskopia direkte,2
2,482,322,2018-04-10,RTG Fillestar,17
3,484,322,2018-04-10,Mikroskopia direkte,2
4,485,323,2018-04-10,Mikroskopia direkte,2
...,...,...,...,...,...
22322,26512,4329,2022-12-02,HIV,2
22323,26513,4329,2022-12-02,GenXpert,8
22324,26514,4329,2022-12-02,Histologjia,3
22325,26515,4329,2022-12-02,TNB-Rezistenca,15


In [42]:
resultConfiguration = {
  'Histologjia': 'RezultatiHistologjia',
  'Mikroskopia direkte': 'RezultatiMikroskopiaDirekte',
  'RTG Fillestar': 'RezultatiRTGFillestar',
  'Kultura': 'RezultatiKultura',
  'GenXpert': 'RezultatiGenXpert',
  'PPD': 'RezultatiPPD',
  'RTG Pasues': 'RezultatiRTGPasues',
  'HIV': 'HIVTestYesNo',
  'Mycobacterium TBC': 'RezultatoMycobacteriumTBC',
  'COVID-19 PCR':  'RezultatiCOVIDPCR',
  'Rtg-CT COVID-19': 'RezultatiCOVIDRtgCT',
  'Serologjik':'RezultatiSerologjik'
}

for row, column in resultConfiguration.items():
    for key, value in configureNominalValues(column).items():
        for index, exam in examinationsDF.iterrows():
            if exam['ExaminationType'] == row and exam['ExaminationResult'] == key:
                examinationsDF.loc[index, 'ExaminationResult'] = value
examinationsDF

,ID,CaseID,RegistrationDate,ExaminationType,ExaminationResult
0,476,320,2018-12-12,Histologjia,Nuk është bërë
1,481,322,2018-04-10,Mikroskopia direkte,Negative
2,482,322,2018-04-10,RTG Fillestar,Infiltrant
3,484,322,2018-04-10,Mikroskopia direkte,Negative
4,485,323,2018-04-10,Mikroskopia direkte,Negative
...,...,...,...,...,...
22322,26512,4329,2022-12-02,HIV,2
22323,26513,4329,2022-12-02,GenXpert,Nuk është bërë
22324,26514,4329,2022-12-02,Histologjia,Nuk është bërë
22325,26515,4329,2022-12-02,TNB-Rezistenca,15


In [43]:
examinationsDF = examinationsDF[examinationsDF['ExaminationResult'].apply(type) != int]

In [44]:
examinationsDF['ExaminationResult'].unique()

array(['Nuk është bërë', 'Negative', 'Infiltrant', 'Normal', 'Pozitive',
       'MTB i pazbuluar/negativ (N)', 'MTB+; RR- (T)', 'Eksudat Pleural',
       'Kavitar', 'MTB+; RR e papercaktuar (TI)', 'MTB+; RR+ (RR)',
       'MTB invalid/Parezultat/Gabim (I)', 'Kontaminuar',
       'Është bërë HIV Test', 'Negativ', 'Pozitiv', 'Jo sugjestiv',
       'Perkeqesim'], dtype=object)

In [45]:
examinationsDF.to_csv('../data/examinations_processed.csv', index=False)